# Calculate Negative

## Features Descriptors

### Preprocessing

#### Initials

##### Installing and calling libraries

In [ ]:
!pip install ifeatpro
!pip install ngrampro
!pip install pssmpro

In [ ]:
def warn(*args, **kwargs):
    pass
import warnings
warnings.warn = warn

In [ ]:
# feature encoding
import pandas as pd
import os
import ifeatpro.features as ipro
import pssmpro.features as ppro
import ngrampro as npro
from scipy import sparse, io
import numpy as np

##### Upload files

Upload the compressed data file, this must contain the training sequences and search space sequences in the 'data/raw/' address. It must also contain the utils folder where the 'Kernels-KeBABs.r' file is located, which will be used to calculate the kernel-based descriptors.

Example: TE.zip

In [ ]:
# Uploded Data
from google.colab import files
import zipfile

uploaded=files.upload()
archivo_zip=list(uploaded.keys())[0]
with zipfile.ZipFile(archivo_zip, 'r') as zip_ref:
    zip_ref.extractall('./')

Saving CYP.zip to CYP.zip


##### Training data

In [ ]:
# change the data names, if you only have one dataset split it into training and search space data
file_train="train_set"
file_test="test_set"
file_dissimilar="dissimilar_set"

In [ ]:
dissimilar_raw="../data/raw/" + file_dissimilar + ".csv"
train_raw="../data/raw/" + file_train + ".csv"
test_raw="../data/raw/" + file_test + ".csv"
df_dissimilar = pd.read_csv(dissimilar_raw, header=None, names=["enz_name", "enz_seq"])
df_train = pd.read_csv(train_raw, header=None, names=["enz_name", "enz_seq", "enz_label"])
df_test = pd.read_csv(test_raw, header=None, names=["enz_name", "enz_seq", "enz_label"])
print(f'The number of enzymatic sequences to train the model is: {len(df_train)}')
print(f'The number of enzymatic sequences to test is: {len(df_test)}')
print(f'The number of enzymatic sequences to test dissimilar is: {len(df_dissimilar)}')
df_dissimilar.head(2)
df_train.head(2)
df_test.head(2)

The number of enzymatic sequences to train the model is: 100
The number of enzymatic sequences to search space is: 100


,enz_name,enz_seq,enz_label
0,sp|P92994|TCMO_ARATH,MDLLLLEKSLIAVFVAVILATVISKLRGKKLKLPPGPIPIPIFGNW...,1
1,sp|O49342|C71AD_ARATH,MEMILSISLCLTTLITLLLLRRFLKRTATKVNLPPSPWRLPVIGNL...,1


##### Raw data analysis
for easier downstream applications

In [ ]:
# parse raw file
# upper case all sequences
def up_seq(seq):
    return seq.upper().replace('-','')


df_train["enz_seq"] = df_train.enz_seq.apply(up_seq)
df_test["enz_seq"] = df_test.enz_seq.apply(up_seq)
df_dissimilar["enz_seq"] = df_dissimilar.enz_seq.apply(up_seq)

# get rid of sequences with illegitimate amino acids
df_train = df_train.loc[~df_train["enz_seq"].str.contains('B|J|O|U|X|Z')]
df_test = df_test.loc[~df_test["enz_seq"].str.contains('B|J|O|U|X|Z')]
df_dissimilar = df_dissimilar.loc[~df_dissimilar["enz_seq"].str.contains('B|J|O|U|X|Z')]

# create enzyme alias
enz_alias_train = [f'enz_{i}' for i in range(len(df_train['enz_name']))]
df_train = df_train.assign(enz_alias=enz_alias_train)
enz_alias_test = [f'test_enz_{i}' for i in range(len(df_test['enz_name']))]
df_test = df_test.assign(enz_alias=enz_alias_test)
enz_alias_dissimilar = [f'dissimilar_enz_{i}' for i in range(len(df_dissimilar['enz_name']))]
df_dissimilar = df_dissimilar.assign(enz_alias=enz_alias_dissimilar)

# enzyme alias to original enzyme name mapping
folder='../data/mappings'
if not os.path.exists(folder):
    os.makedirs(folder)
enz_train_name_map = folder + "/train_enz_map.csv"
enz_test_name_map = folder + "/test_enz_map.csv"
enz_dissimilar_name_map = folder + "/dissimilar_enz_map.csv"
df_train.loc[:, ["enz_alias", "enz_name"]].to_csv(enz_train_name_map, index=False, header=False)
df_test.loc[:, ["enz_alias", "enz_name"]].to_csv(enz_test_name_map, index=False, header=False)
df_dissimilar.loc[:, ["enz_alias", "enz_name"]].to_csv(enz_dissimilar_name_map, index=False, header=False)

# names in fasta
folder='../data/seq'
if not os.path.exists(folder):
    os.makedirs(folder)
enz_train_fasta = folder + "/train_enz.fa"
enz_test_fasta = folder + "/test_enz.fa"
enz_dissimilar_fasta = folder + "/dissimilar_enz.fa"


train_fasta_stream = open(enz_train_fasta, 'w')
test_fasta_stream = open(enz_test_fasta, 'w')
dissimilar_fasta_stream = open(enz_dissimilar_fasta, "w")

for value in df_train.loc[:, ["enz_alias", "enz_seq"]].values:
    train_fasta_stream.write(f">{value[0]}\n{value[1]}\n")

for value in df_test.loc[:, ["enz_alias", "enz_seq"]].values:
    test_fasta_stream.write(f">{value[0]}\n{value[1]}\n")

for value in df_dissimilar.loc[:, ["enz_alias", "enz_seq"]].values:
    dissimilar_fasta_stream.write(f">{value[0]}\n{value[1]}\n")

train_fasta_stream.close()
test_fasta_stream.close()
dissimilar_fasta_stream.close()

# create csv file of sequence
enz_train_csv = folder + "/train_enz.csv"
enz_dissimilar_csv = folder + "/dissimilar_enz.csv"
enz_test_csv = folder + "/test_enz.csv"
df_train.loc[:, ["enz_alias", "enz_seq"]].to_csv(enz_train_csv, header=False, index=False)
df_test.loc[:, ["enz_alias", "enz_seq"]].to_csv(enz_test_csv, header=False, index=False)
df_dissimilar.loc[:, ["enz_alias", "enz_seq"]].to_csv(enz_dissimilar_csv, header=False, index=False)

##### Creating labels

In [ ]:
# create labels as csv
folder='../data/label'
if not os.path.exists(folder):
    os.makedirs(folder)
train_labels = folder + "/train_enz_label.csv"
test_labels = folder + "/test_enz_label.csv"
dissimilar_labels = folder + "/dissimilar_enz_label.csv"
df_train.loc[:, ["enz_alias", "enz_label"]].to_csv(train_labels, index=False, header=False)
df_test.loc[:, ["enz_alias", "enz_label"]].to_csv(test_labels, index=False, header=False)
df_dissimilar.loc[:, ["enz_alias", "enz_label"]].to_csv(dissimilar_labels, index=False, header=False)

### Numerical coding of sequences (Descriptors)


The sequences are numerically encoded in 47 different ways. They are:

21 types of physicochemical encodings using the [ifeatpro](https://github.com/deeprob/ifeatpro).

21 types of PSSM-based encodings using the [pssmpro](https://pypi.org/project/pssmpro/).

2 types of ngram-based encodings using the [ngrampro](https://pypi.org/project/ngrampro/).

3 types of kernel-based encodings using an external tool written in R known as [KeBABS](https://bioconductor.org/packages/release/bioc/vignettes/kebabs/inst/doc/kebabs.pdf).



#### [Ifeatpro](https://github.com/deeprob/ifeatpro)
The ifeatpro link can be used directly with a fasta file containing protein sequences in fasta format.

In [ ]:
help(ipro.get_all_features)

Help on function get_all_features in module ifeatpro.features:

get_all_features(fasta_file, output_dir)
    A function to create 21 numerically encoded features for protein sequences
    :param fasta_file: The path to a file that contains all the protein sequences in fasta format
    :param output_dir: The path to a directory where the feature encoded files will be stored
    :return: None



In [ ]:
%%time
# Addresses
folder_data = '../data/seq'
train_fasta_file = folder_data + "train_enz.fa"
test_fasta_file = folder_data + "test_enz.fa"
dissimilar_fasta_file = folder_data + "dissimilar_enz.fa"

folder = '../features/enzymes/descriptors'

train_output_dir = folder + "/ifeatpro/train/"
test_output_dir = folder + "/ifeatpro/test/"
dissimilar_output_dir = folder + "/ifeatpro/dissimilar/"

os.makedirs(train_output_dir, exist_ok=True)
os.makedirs(test_output_dir, exist_ok=True)
os.makedirs(dissimilar_output_dir, exist_ok=True)

Descriptor type: aac
Descriptor type: cksaap
Descriptor type: tpc
Descriptor type: dpc
Descriptor type: dde
Descriptor type: gaac
Descriptor type: cksaagp
Descriptor type: gtpc
Descriptor type: gdpc
Descriptor type: moran
Descriptor type: geary
Descriptor type: nmbroto
Descriptor type: ctdc
Descriptor type: ctdt
Descriptor type: ctdd
Descriptor type: ctriad
Descriptor type: ksctriad
Descriptor type: socnumber
Descriptor type: qsorder
Descriptor type: paac
Descriptor type: apaac
Descriptor type: aac
Descriptor type: cksaap
Descriptor type: tpc
Descriptor type: dpc
Descriptor type: dde
Descriptor type: gaac
Descriptor type: cksaagp
Descriptor type: gtpc
Descriptor type: gdpc
Descriptor type: moran
Descriptor type: geary
Descriptor type: nmbroto
Descriptor type: ctdc
Descriptor type: ctdt
Descriptor type: ctdd
Descriptor type: ctriad
Descriptor type: ksctriad
Descriptor type: socnumber
Descriptor type: qsorder
Descriptor type: paac
Descriptor type: apaac


In [ ]:
%%time
print(f'****************** train: {len(df_train)} sequences ***************************')
ipro.get_all_features(train_fasta_file, train_output_dir)
print(f'****************** test: {len(df_test)} sequences ***************************')
ipro.get_all_features(test_fasta_file, test_output_dir)
print(f'****************** dissimilar: {len(df_dissimilar)} sequences ***************************')
ipro.get_all_features(dissimilar_fasta_file, dissimilar_output_dir)

#### [Pssmpro](https://pypi.org/project/pssmpro/)

Pssmpro requires the pssm profile of protein sequences as input. At first, it is necessary to create the pssm profiles and then these profiles can be numerically encoded using the function provided by pssmpro. pssmpro also provides a function to create numerical encodings of protein sequences. The psiblast program path and an indexed blast database are required as function argument.



##### Creating pssm profiles

In [ ]:
help(ppro.create_pssm_profile)

Help on function create_pssm_profile in module pssmpro.features:

create_pssm_profile(seq_file, out_dir, psiblast_exec, database_prefix, num_threads=24)
    A function to create psiblast or pssm profile for protein sequences
    :param seq_file: A csv file with name of the protein followed by its sequence separated by a comma
    :param out_dir: The directory where the user would like to store the pssm profiles of all the sequences
    :param psiblast_exec: The path of the psiblast executable. psiblast program needs to be installed
    :param database_prefix: The path of the indexed blast database directory prefix
    :param num_threads: Number of threads to use while creating the psiblast profile
    :return: The output directory where the psiblast/pssm profiles are stored



###### Installation Blast and download database swissprot

In [ ]:
!pip install biopython #Install blast package
#download database to make blast
#linux, colab or macOS
!wget https://ftp.ncbi.nlm.nih.gov/blast/db/FASTA/swissprot.gz # linux or colab
#!wget ftp://ftp.uniprot.org/pub/databases/uniprot/current_release/knowledgebase/complete/uniprot_sprot.fasta.gz

#windows
#import urllib.request
#url = "https://ftp.ncbi.nlm.nih.gov/blast/db/FASTA/swissprot.gz"
#output_file = "swissprot.gz"
#urllib.request.urlretrieve(url, output_file)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 26.9 MB/s eta 0:00:00
--2025-05-05 18:22:42--  https://ftp.ncbi.nlm.nih.gov/blast/db/FASTA/swissprot.gz
Resolving ftp.ncbi.nlm.nih.gov (ftp.ncbi.nlm.nih.gov)... 130.14.250.7, 130.14.250.10, 130.14.250.11, ...
Connecting to ftp.ncbi.nlm.nih.gov (ftp.ncbi.nlm.nih.gov)|130.14.250.7|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 143234298 (137M) [application/x-gzip]
Saving to: ‘swissprot.gz’

swissprot.gz        100%[===================>] 136.60M  65.9MB/s    in 2.1s    

2025-05-05 18:22:44 (65.9 MB/s) - ‘swissprot.gz’ saved [143234298/143234298]



In [ ]:
# linux or colab
!gunzip swissprot.gz
#!gunzip uniprot_sprot.fasta.gz
# windows
#import gzip
#import shutil

#with gzip.open('swissprot.gz', 'rb') as f_in:
#    with open('swissprot', 'wb') as f_out:
#        shutil.copyfileobj(f_in, f_out)

In [ ]:
# We install blast in the environment
!apt-get install ncbi-blast+
# You can also download the executables and run them.
#!wget ftp://ftp.ncbi.nlm.nih.gov/blast/executables/blast+/2.9.0/ncbi-blast-2.9.0+-x64-linux.tar.gz
# Unzip either of the two
#!tar -zxvf ncbi-blast-2.9.0+-x64-linux.tar.gz
#!gunzip ncbi-blast-2.9.0+-x64-linux.tar.gz
# We add it to the directory so that it can be executed
#os.environ['PATH'] += ":/content/ncbi-blast-2.9.0+/bin"

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  ncbi-data
The following NEW packages will be installed:
  ncbi-blast+ ncbi-data
0 upgraded, 2 newly installed, 0 to remove and 34 not upgraded.
Need to get 15.8 MB of archives.
After this operation, 71.8 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 ncbi-data all 6.1.20170106+dfsg1-9 [3,519 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/universe amd64 ncbi-blast+ amd64 2.12.0+ds-3build1 [12.3 MB]
Fetched 15.8 MB in 1s (13.1 MB/s)
Selecting previously unselected package ncbi-data.
(Reading database ... 126101 files and directories currently installed.)
Preparing to unpack .../ncbi-data_6.1.20170106+dfsg1-9_all.deb ...
Unpacking ncbi-data (6.1.20170106+dfsg1-9) ...
Selecting previously unselected package ncbi-blast+.
Preparing to unpack .../ncbi-blast+_2.12.0+ds-3build1_amd64.deb .

In [ ]:
# We generate the database for blast from the downloaded one
os.rename('swissprot', 'swissprot.fasta')
#!makeblastdb -in uniprot_sprot.fasta -dbtype prot -out ./db/sprot
!makeblastdb -in swissprot.fasta -dbtype prot -out ./db/swissprot



Building a new DB, current time: 05/05/2025 18:23:10
New DB name:   /content/db/swissprot
New DB title:  swissprot.fasta
Sequence type: Protein
Keep MBits: T
Maximum file size: 1000000000B
Adding sequences from FASTA; added 482697 sequences in 18.7495 seconds.




###### Generating psiblast profile

In [ ]:
%%time
#database_pre = "db/sprot"
database_pre = "./db/swissprot"
train_seq_file = "../data/seq/train_enz.csv"
test_seq_file = "../data/seq/test_enz.csv"
dissimilar_seq_file = "../data/seq/dissimilar_enz.csv"
psiblast_path = "/usr/bin/psiblast"
output_dir_train = "../features/descriptors/pssmpro/pssm_profiles/train/"
os.makedirs(output_dir_train, exist_ok=True)
ppro.create_pssm_profile(train_seq_file, output_dir_train, psiblast_path, database_pre)

Generating psiblast profile for protein: enz_0
Generating psiblast profile for protein: enz_1
Generating psiblast profile for protein: enz_2
Generating psiblast profile for protein: enz_3
Generating psiblast profile for protein: enz_4
Generating psiblast profile for protein: enz_5
Generating psiblast profile for protein: enz_6
Generating psiblast profile for protein: enz_7
Generating psiblast profile for protein: enz_8
Generating psiblast profile for protein: enz_9
Generating psiblast profile for protein: enz_10
Generating psiblast profile for protein: enz_11
Generating psiblast profile for protein: enz_12
Generating psiblast profile for protein: enz_13
Generating psiblast profile for protein: enz_14
Generating psiblast profile for protein: enz_15
Generating psiblast profile for protein: enz_16
Generating psiblast profile for protein: enz_17
Generating psiblast profile for protein: enz_18
Generating psiblast profile for protein: enz_19
Generating psiblast profile for protein: enz_20
Ge

'./analysis/features/pssmpro/pssm_profiles/train/'

In [ ]:
%%time
output_dir_test = "../features/descriptors/pssmpro/pssm_profiles/test/"
os.makedirs(output_dir_test, exist_ok=True)
ppro.create_pssm_profile(test_seq_file, output_dir_test, psiblast_path, database_pre)

Generating psiblast profile for protein: search_space_enz_0
Generating psiblast profile for protein: search_space_enz_1
Generating psiblast profile for protein: search_space_enz_2
Generating psiblast profile for protein: search_space_enz_3
Generating psiblast profile for protein: search_space_enz_4
Generating psiblast profile for protein: search_space_enz_5
Generating psiblast profile for protein: search_space_enz_6
Generating psiblast profile for protein: search_space_enz_7
Generating psiblast profile for protein: search_space_enz_8
Generating psiblast profile for protein: search_space_enz_9
Generating psiblast profile for protein: search_space_enz_10
Generating psiblast profile for protein: search_space_enz_11
Generating psiblast profile for protein: search_space_enz_12
Generating psiblast profile for protein: search_space_enz_13
Generating psiblast profile for protein: search_space_enz_14
Generating psiblast profile for protein: search_space_enz_15
Generating psiblast profile for pr

In [ ]:
%%time
output_dir_dissimilar = "../features/descriptors/pssmpro/pssm_profiles/dissimilar/"
os.makedirs(output_dir_dissimilar, exist_ok=True)
ppro.create_pssm_profile(dissimilar_seq_file, output_dir_dissimilar, psiblast_path, database_pre)

###### Generating features from profiles.

In [ ]:
help(ppro.get_all_features)

Help on function get_all_features in module pssmpro.features:

get_all_features(pssm_dir, store_dir='./')



In [ ]:
train_out_dir = "../features/descriptors/pssmpro/train/"
test_out_dir = "../features/descriptors/pssmpro/test/"
dissimilar_out_dir = "../features/descriptors/pssmpro/dissimilar/"

In [ ]:
%%time
# It takes a while to generate the profiles
os.makedirs(train_out_dir, exist_ok=True)
ppro.get_all_features("../features/descriptors/pssmpro/pssm_profiles/train/", train_out_dir)

['aac_pssm',
 'aadp_pssm',
 'aatp',
 'ab_pssm',
 'd_fpssm',
 'dp_pssm',
 'dpc_pssm',
 'edp',
 'eedp',
 'k_separated_bigrams_pssm',
 'medp',
 'pse_pssm',
 'pssm_ac',
 'pssm_cc',
 'pssm_composition',
 'rpm_pssm',
 'rpssm',
 's_fpssm',
 'smoothed_pssm',
 'tpc_pssm',
 'tri_gram_pssm']

In [ ]:
%%time
os.makedirs(test_out_dir, exist_ok=True)
ppro.get_all_features("../features/descriptors/pssmpro/pssm_profiles/test/", test_out_dir)

['aac_pssm',
 'aadp_pssm',
 'aatp',
 'ab_pssm',
 'd_fpssm',
 'dp_pssm',
 'dpc_pssm',
 'edp',
 'eedp',
 'k_separated_bigrams_pssm',
 'medp',
 'pse_pssm',
 'pssm_ac',
 'pssm_cc',
 'pssm_composition',
 'rpm_pssm',
 'rpssm',
 's_fpssm',
 'smoothed_pssm',
 'tpc_pssm',
 'tri_gram_pssm']

In [ ]:
%%time
os.makedirs(dissimilar_out_dir, exist_ok=True)
ppro.get_all_features("../features/descriptors/pssmpro/pssm_profiles/dissimilar/", dissimilar_out_dir)

#### [Ngrampro](https://pypi.org/project/ngrampro/)




In [ ]:
from ngrampro import NGModel, GAANGModel

In [ ]:
train_enz_seq = "../data/seq/train_enz.csv"
label_file = "../data/label/train_enz_label.csv"


train_output_dir = "../features/descriptors/ifeatpro/train/"
test_output_dir = "../features/descriptors/ifeatpro/test/"
dissimilar_output_dir = "../features/descriptors/ifeatpro/dissimilar/"

# create output directories
kmer_train_dir = "./analysis/features/ngrampro/kmer/train/"
kmer_search_space_dir = "./analysis/features/ngrampro/kmer/search_space/"
gaakmer_train_dir = "./analysis/features/ngrampro/gaakmer/train/"
gaakmer_search_space_dir = "./analysis/features/ngrampro/gaakmer/search_space/"

os.makedirs(kmer_train_dir, exist_ok=True)
os.makedirs(kmer_search_space_dir, exist_ok=True)
os.makedirs(gaakmer_train_dir, exist_ok=True)
os.makedirs(gaakmer_search_space_dir, exist_ok=True)

In [ ]:
# Train
df1 = pd.read_csv(train_enz_seq, header=None)
df2 = pd.read_csv(label_file, header=None)
train_df = df1.merge(df2, on=0)
enz_names_train = train_df[0].values
X_train = train_df.iloc[:, 1].values

In [ ]:
# test
test_enz_seq = "../data/seq/test_enz.csv"
test_df = pd.read_csv(test_enz_seq, header=None)
enz_names_test = test_df[0].values
X_test = test_df.iloc[:, 1].values

In [ ]:
# dissimilar
dissimilar_enz_seq = "../data/seq/dissimilar_enz.csv"
dissimilar_df = pd.read_csv(dissimilar_enz_seq, header=None)
enz_names_dissimilar = dissimilar_df[0].values
X_dissimilar = dissimilar_df.iloc[:, 1].values

In [ ]:
%%time
# Calculate descriptors with NGModel and GAANGModel
# NGModel needs X_train, X_valid, X_test, but since we don't want splitting, we pass an empty array for X_valid
ng = NGModel(X_train, np.array([]), X_test)
gaang = GAANGModel(X_train, np.array([]), X_test)

In [ ]:
%%time
# NGModel (kmer)
ng_train_df = pd.DataFrame(ng.x_train, index=enz_names_train)
ng_train_df.to_csv(os.path.join(kmer_train_dir, "kmer_train.csv"))

ng_test_df = pd.DataFrame(ng.x_test, index=enz_names_test)
ng_test_df.to_csv(os.path.join(kmer_test_dir, "kmer_test.csv"))

ng_dissimilar_df = pd.DataFrame(ng.x_dissimilar, index=enz_names_dissimilar)
ng_dissimilar_df.to_csv(os.path.join(kmer_dissimilar_dir, "kmer_dissimilar.csv"))

In [ ]:
%%time
# GAANGModel (gaakmer)
gaang_train_df = pd.DataFrame(gaang.x_train, index=enz_names_train)
gaang_train_df.to_csv(os.path.join(gaakmer_train_dir, "gaakmer_train.csv"))

gaang_test_df = pd.DataFrame(gaang.x_test, index=enz_names_test)
gaang_test_df.to_csv(os.path.join(gaakmer_test_dir, "gaakmer_test.csv"))

gaang_dissimilar_df = pd.DataFrame(gaang.x_dissimilar, index=enz_names_dissimilar)
gaang_dissimilar_df.to_csv(os.path.join(gaakmer_dissimilar_dir, "gaakmer_dissimilar.csv"))

#### [KeBABS](https://bioconductor.org/packages/release/bioc/vignettes/kebabs/inst/doc/kebabs.pdf)

Kernel-based functions are created using an external software package called KeBABS. The Rscript used to generate the features called Kernels-KeBABs.r is located in the utils directory.


##### Run the R script

In [ ]:
# We install Kebabs on the version of R that runs colab
!R -e "if (!requireNamespace('BiocManager', quietly = TRUE)) install.packages('BiocManager')"
!R -e "BiocManager::install('kebabs')"


R version 4.5.0 (2025-04-11) -- "How About a Twenty-Six"
Copyright (C) 2025 The R Foundation for Statistical Computing
Platform: x86_64-pc-linux-gnu

R is free software and comes with ABSOLUTELY NO WARRANTY.
You are welcome to redistribute it under certain conditions.
Type 'license()' or 'licence()' for distribution details.

  Natural language support but running in an English locale

R is a collaborative project with many contributors.
Type 'contributors()' for more information and
'citation()' on how to cite R or R packages in publications.

Type 'demo()' for some demos, 'help()' for on-line help, or
'help.start()' for an HTML browser interface to help.
Type 'q()' to quit R.

> if (!requireNamespace('BiocManager', quietly = TRUE)) install.packages('BiocManager')
Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)
trying URL 'https://cran.rstudio.com/src/contrib/BiocManager_1.30.25.tar.gz'
Content type 'application/x-gzip' length 593414 bytes (579 KB)
d

In [ ]:
# We create the folders to save the descriptors
dir = '../features/descriptors/kernel/spec/'
if not os.path.exists(dir):
    os.makedirs(dir)

dir = '../features/descriptors/kernel/mism/'
if not os.path.exists(dir):
    os.makedirs(dir)

dir = '../features/descriptors/kernel/gap/'
if not os.path.exists(dir):
    os.makedirs(dir)

In [ ]:
# Running the Rscript using a bash command
!Rscript ./utils/Kernels-KeBABs.r

Loading required package: Biostrings
Loading required package: BiocGenerics
Loading required package: generics

Attaching package: ‘generics’

The following objects are masked from ‘package:base’:

    as.difftime, as.factor, as.ordered, intersect, is.element, setdiff,
    setequal, union


Attaching package: ‘BiocGenerics’

The following objects are masked from ‘package:stats’:

    IQR, mad, sd, var, xtabs

The following objects are masked from ‘package:base’:

    anyDuplicated, aperm, append, as.data.frame, basename, cbind,
    colnames, dirname, do.call, duplicated, eval, evalq, Filter, Find,
    get, grep, grepl, is.unsorted, lapply, Map, mapply, match, mget,
    order, paste, pmax, pmax.int, pmin, pmin.int, Position, rank,
    rbind, Reduce, rownames, sapply, saveRDS, table, tapply, unique,
    unsplit, which.max, which.min

Loading required package: S4Vectors
Loading required package: stats4

Attaching package: ‘S4Vectors’

The following object is masked from ‘package:utils’:



##### Analyzing the exit of Kebabs


In [ ]:
featfile_dir = "../features/descriptors/kernel/"
file_prefixes = ['spec', 'gap', 'mism']
output_train_file = "../features/descriptors/kernel/train/"
output_test_file = "../features/descriptors/kernel/test/"
output_dissimilar_file = "../features/descriptors/kernel/dissimilar/"

os.makedirs(output_train_file, exist_ok=True)
os.makedirs(output_test_file, exist_ok=True)
os.makedirs(output_dissimilar_file, exist_ok=True)

In [ ]:
def save_feat_vec_files(file_dir, outdir_train, outdir_search_space, file_prefix):
    '''
      It takes the kernel features (mismatch, spectrum, and Gappy) and saves them in the train and search_space folders.

      Inputs
        file_dir: String; Path to the directory where the input files are located.
        outdir_train: String; Path to the directory where the output files for the training enzymes will be saved.
        outdir_search_space: String; Path to the directory where the output files for the search space enzymes will be saved.
        file_prefix: String; Prefix used to construct the names of the input and output files.
    '''

    sp_mat_file = file_dir + file_prefix + "/" + file_prefix + '_kern_sparsematrix.txt'
    enz_name_file = file_dir + file_prefix + "/" + file_prefix + '_kern_rownames.txt'
    print(sp_mat_file)
    print(enz_name_file)
    sp_mat = io.mmread(sp_mat_file).tocsr()
    enz_names = np.genfromtxt(enz_name_file, dtype=str)


    train_enz_idx = []
    search_space_enz_idx = []

    for idx, enz_name in enumerate(enz_names):
        if enz_name.startswith('enz'):
            train_enz_idx.append(idx)
        elif enz_name.startswith('search_space'):
            search_space_enz_idx.append(idx)
        else:
            raise ValueError('Wrong Enzyme Prefix')

    X_train, X_search_space = sp_mat[train_enz_idx,:], sp_mat[search_space_enz_idx,:]

    enz_names_train, enz_names_search_space = enz_names[train_enz_idx], enz_names[search_space_enz_idx]

    assert X_train.shape[0] == len(enz_names_train)
    assert X_search_space.shape[0] == len(enz_names_search_space)


    sparse.save_npz(outdir_train+file_prefix+'mat.npz', X_train)
    sparse.save_npz(outdir_search_space+file_prefix+'mat.npz', X_search_space)

    np.savetxt(outdir_train+file_prefix+'enz_names.txt', enz_names_train, fmt='%s')
    np.savetxt(outdir_search_space+file_prefix+'enz_names.txt', enz_names_search_space, fmt='%s')


    return

In [ ]:
for fp in file_prefixes:
    save_feat_vec_files(featfile_dir, output_train_file,
                        output_search_space_file, fp)

./analysis/features/kernel/spec/spec_kern_sparsematrix.txt
./analysis/features/kernel/spec/spec_kern_rownames.txt
./analysis/features/kernel/gap/gap_kern_sparsematrix.txt
./analysis/features/kernel/gap/gap_kern_rownames.txt
./analysis/features/kernel/mism/mism_kern_sparsematrix.txt
./analysis/features/kernel/mism/mism_kern_rownames.txt
